# Test of the Optimization function elaborated

In [1]:
import numpy as np
import pandas as pd
import control as ct
import control.optimal as opt
import matplotlib.pyplot as plt

# from analysis_irrigation import IrrigationAnalysis

# Trying out by the beggining.
from DSSATTools import (
    Crop, SoilProfile, WeatherData, WeatherStation,
    Management, DSSAT, TabularSubsection
)
from datetime import datetime, timedelta

## Test 01 - Basics

In [ ]:
# Random weather data
DATES = pd.date_range('2000-01-01', '2002-12-31')
N = len(DATES)
df = pd.DataFrame(
    {
    'tn':   np.random.gamma(10, 1, N),
    'rad':  np.random.gamma(10, 1.5, N),
    'prec': [0.0]* N,
    'rh':   100 * np.random.beta(1.5, 1.15, N),
    },
    index=DATES,
)
df['TMAX'] = df.tn + np.random.gamma(5., .5, N)
df.head()

In [ ]:
# Create a WeatherData instance
WTH_DATA = WeatherData(
    df,
    variables={
        'tn': 'TMIN', 'TMAX': 'TMAX',
        'prec': 'RAIN', 'rad': 'SRAD',
        'rh': 'RHUM'
    }
)
# Create a WheaterStation instance
wth = WeatherStation(
    WTH_DATA, 
    {'ELEV': 33, 'LAT': 0, 'LON': 0, 'INSI': 'dpoes'}
)

In [ ]:
# Soil instance from default soil profile
soil = SoilProfile(default_class='SIC')

# Crop
crop = Crop('potato')
# Check how the cultivar looks like
crop.cultivar['IB0001']

In [ ]:
# Creating irrigation schedule
schedule = pd.DataFrame([
    (datetime(2000,1,15), 80),
    (datetime(2000,1,30), 60),
    (datetime(2000,2,15), 40),
    (datetime(2000,3,1),  20)
], columns = ['date', 'IRVAL'])
schedule['IDATE'] = schedule.date.dt.strftime('%y%j')
schedule['IROP'] = 'IR001' # Code to define the irrigation operation
schedule = schedule[['IDATE', 'IROP', 'IRVAL']]
schedule

In [ ]:
# Management instance
man = Management(
    cultivar='IB0001',
    planting_date=DATES[10],
)
# Modify harvest to Maturity
man.simulation_controls['HARVS'] = 'M'
# Set the irrigation schedule
man.irrigation['table'] = TabularSubsection(schedule)
# Set irrigation control as reported schedule
man.simulation_controls['IRRIG'] = 'R'

# The definition of this parameters is mandatory for roots (???)
man.planting_details['table']['PLWT'] = 1500
man.planting_details['table']['SPRL'] = 2

# Check the simulation control value for Irrigation
man.simulation_controls['IRRIG']

In [ ]:
dssat = DSSAT()
dssat.setup()
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)

In [ ]:
# Modify the irrigation amounts and check output
schedule['IRVAL'] = [50, 50, 50, 50]
man.irrigation['table'] = TabularSubsection(schedule)
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)

In [ ]:
# Print available outputs
dssat.OUTPUT_LIST

# Save the output
output_1 = dssat.output['PlantGro']
output_1.head()

In [ ]:
# Terminate the DSSAT instance
dssat.close()

## Test 02 - Scipy Optimization

In [ ]:
from optimization import optimization_irri
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [ ]:
###### SETUP OF THE PROBLEM

# Random weather data
DATES = pd.date_range('2000-01-01', '2000-02-28')
N = len(DATES)
df = pd.DataFrame(
    {
    'tn':   np.random.gamma(10, 1, N),
    'rad':  np.random.gamma(10, 1.5, N),
    'prec': [0.0]* N,
    'rh':   100 * np.random.beta(1.5, 1.15, N),
    },
    index=DATES,
)
df['TMAX'] = df.tn + np.random.gamma(5., .5, N)

# Create a WeatherData instance
WTH_DATA = WeatherData(
                        df,
                        variables={
                        'tn': 'TMIN', 'TMAX': 'TMAX',
                        'prec': 'RAIN', 'rad': 'SRAD',
                        'rh': 'RHUM'
                    }
)
# Create a WheaterStation instance
wth = WeatherStation(
    WTH_DATA, 
    {'ELEV': 33, 'LAT': 0, 'LON': 0, 'INSI': 'dpoes'}
)

# Soil instance from default soil profile
soil = SoilProfile(default_class='SIC')
# Crop
crop = Crop('potato')
# Check how the cultivar looks like
crop.cultivar['IB0001']

# Management instance
man = Management(
    cultivar='IB0001',
    planting_date=DATES[10],
)
# Modify harvest to Maturity
man.simulation_controls['HARVS'] = 'M'
# Set irrigation control as reported schedule
man.simulation_controls['IRRIG'] = 'R'

# The definition of this parameters is mandatory for roots (???)
man.planting_details['table']['PLWT'] = 1500
man.planting_details['table']['SPRL'] = 2

In [ ]:
irval = [0,  0, 0 , 0 ]
optimization_irri(irval)

In [ ]:
# Definition of the optimization problem

bounds = Bounds([2,2,2,2], [30,30,30,30])

irval0 = np.array([0, 0, 0, 0])
res = minimize(optimization_irri, irval0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})

In [ ]:
print(res.x)

## Test 03 - Nevergrad

In [ ]:
from optimization import optimization_irri
import numpy as np
import nevergrad as ng

In [3]:
###### SETUP OF THE PROBLEM

# Random weather data
DATES = pd.date_range('2000-01-01', '2000-02-28')
N = len(DATES)
df = pd.DataFrame(
    {
    'tn':   np.random.gamma(10, 1, N),
    'rad':  np.random.gamma(10, 1.5, N),
    'prec': [0.0]* N,
    'rh':   100 * np.random.beta(1.5, 1.15, N),
    },
    index=DATES,
)
df['TMAX'] = df.tn + np.random.gamma(5., .5, N)

# Create a WeatherData instance
WTH_DATA = WeatherData(
                        df,
                        variables={
                        'tn': 'TMIN', 'TMAX': 'TMAX',
                        'prec': 'RAIN', 'rad': 'SRAD',
                        'rh': 'RHUM'
                    }
)
# Create a WheaterStation instance
wth = WeatherStation(
    WTH_DATA, 
    {'ELEV': 33, 'LAT': 0, 'LON': 0, 'INSI': 'dpoes'}
)

# Soil instance from default soil profile
soil = SoilProfile(default_class='SIC')
# Crop
crop = Crop('potato')
# Check how the cultivar looks like
crop.cultivar['IB0001']

# Management instance
man = Management(
    cultivar='IB0001',
    planting_date=DATES[10],
)
# Modify harvest to Maturity
man.simulation_controls['HARVS'] = 'M'
# Set irrigation control as reported schedule
man.simulation_controls['IRRIG'] = 'R'

# The definition of this parameters is mandatory for roots (???)
man.planting_details['table']['PLWT'] = 1500
man.planting_details['table']['SPRL'] = 2

In [4]:
max_val_irrig = 60
irval = [max_val_irrig, max_val_irrig, max_val_irrig, max_val_irrig]
optimization_irri(irval)

C:\Users\leopo\AppData\Local\Temp\dssatlwxgawav created.
Static files copied to C:\Users\leopo\AppData\Local\Temp\dssatlwxgawav.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  28 -99  4628  2541     0   240   125   218     0   -99      0  212


1.4183075555555555

In [5]:
# Definition of the optimization problem

# Discrete, ordered
param = ng.p.TransitionChoice(range(max_val_irrig), repetitions=4)
optimizer = ng.optimizers.DiscreteOnePlusOne(parametrization=param, budget=100)
# optimizer = ng.optimizers.DiscreteOnePlusOne(parametrization=param, budget=100, num_workers=1)

recommendation = optimizer.provide_recommendation()
for _ in range(optimizer.budget):
    x = optimizer.ask()
    loss = optimization_irri(x.value)
    optimizer.tell(x, loss)

recommendation = optimizer.provide_recommendation()
print(recommendation.value)

C:\Users\leopo\AppData\Local\Temp\dssatjvjgkacc created.
Static files copied to C:\Users\leopo\AppData\Local\Temp\dssatjvjgkacc.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  28 -99  4628  2541     0   120   123   102     0   -99      0  212
C:\Users\leopo\AppData\Local\Temp\dssatmcycbcbc created.
Static files copied to C:\Users\leopo\AppData\Local\Temp\dssatmcycbcbc.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  28 -99  4605  2483     0   100   123    83     0   -99      0  212
C:\Users\leopo\AppData\Local\Temp\dssativlnathw created.
Static files copied to C:\Users\leopo\AppData\Local\Temp\dssativlnathw.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha

In [11]:
# Test of the value found
# irval = [30, 36, 18, 3] - loss: 0.5546 (optimal found)
# irval = [36, 30, 10, 2] - loss: 0.78297 
# irval = [30, 30, 10, 7] - loss: 0.833
irval = [31, 5, 30, 10]
optimization_irri(irval)

C:\Users\leopo\AppData\Local\Temp\dssatsqjnaznt created.
Static files copied to C:\Users\leopo\AppData\Local\Temp\dssatsqjnaznt.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  28 -99  3429  2025     0    76   114    67     0   -99      0  212


1.5691757777777777

In [ ]:
# Definition of the optimization problem

# Parametrization
instrum = ng.p.Instrumentation(
    ng.p.Array(shape=(4,)).set_bounds(lower=0, upper=50)
)

# optimization on x as an array of shape (4,) (instrum)
optimizer = ng.optimizers.NGOpt(parametrization=instrum, budget=100)
recommendation = optimizer.minimize(optimization_irri, verbosity=5)

# optimization on x as an array of shape (4,)  ->>> What is the budget?
# optimizer = ng.optimizers.NGOpt(parametrization=4, budget=100)

# define a constraint on first variable of x:
# optimizer.parametrization.register_cheap_constraint(lambda x: x[0] >= 0)
# optimizer.parametrization.register_cheap_constraint(lambda x: x[1] >= 0)
# optimizer.parametrization.register_cheap_constraint(lambda x: x[2] >= 0)
# optimizer.parametrization.register_cheap_constraint(lambda x: x[3] >= 0)

# What is verbosity?
# recommendation = optimizer.minimize(optimization_irri, verbosity=4)  # best value

print(recommendation.value)